# Boltzmann machine

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Jul 13 20:27:11 2018

@author: NVDL

Boltzmann machine to predict rating of movies based on customer ratings.
"""
###Part 1 - Importing 
#Importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
#Importing the dataset
movies = pd.read_csv('/Users/NVDL/Code/Practice_/Data/Math/Ranking/Movies/ml-1m/movies.dat',
                     sep='::',
                     header= None, 
                     engine = 'python', 
                     encoding ='latin-1')

users = pd.read_csv('/Users/NVDL/Code/Practice_/Data/Math/Ranking/Movies/ml-1m/users.dat',
                     sep='::',
                     header= None, 
                     engine = 'python', 
                     encoding ='latin-1')

ratings = pd.read_csv('/Users/NVDL/Code/Practice_/Data/Math/Ranking/Movies/ml-1m/ratings.dat',
                     sep='::',
                     header= None, 
                     engine = 'python', 
                     encoding ='latin-1')

In [3]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [6]:
### Part 2 - Preprocessing train/test set
#Preparing the training set 
training_set = pd.read_csv('/Users/NVDL/Code/Practice_/Data/Math/Ranking/Movies/ml-100k/u1.base',
                           delimiter = '\t') #80% of total set 
#Convert df training_set to array
training_set = np.array(training_set, dtype = 'int')

#Preparing the the test set
test_set = pd.read_csv('/Users/NVDL/Code/Practice_/Data/Math/Ranking/Movies/ml-100k/u1.test',
                           delimiter = '\t') #20% of total set 
#Convert df test_set to array
test_set = np.array(test_set, dtype = 'int')


In [7]:
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [8]:
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

Add variable to confirm # of customers and # of movies. To do this we add the 
maximum values of the columns of the training_set and test_set. 

In [9]:
#Getting the number of users and movies
num_users = int(max(max(training_set[:,0]), max(test_set[:,0]))) #first column, maximum user id
num_movies = int(max(max(training_set[:,1]), max(test_set[:,1]))) #first column, no. of movies

In [10]:
#Converting data into array with users in lines 
#and movies in columns (943 list of users with list of 1682 movies usersrating)
def convert(data):
    new_data = []
    for id_users in range(1, num_users + 1): #1, up to [shift data 1+] num_users 
        id_movies = data[:,1][data[:,0] == id_users] #movie of users
        id_ratings = data[:,2][data[:,0] == id_users] #movie rating of users
        ratings = np.zeros(num_movies)
        ratings[id_movies - 1] = id_ratings 
        new_data.append(list(ratings))
    return new_data 

In [11]:
#Convert training set to return list of customers with lists of rating per movie
training_set = convert(training_set)
test_set = convert(test_set)

In [12]:
#Converting the dataset into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)    

#Converting ratings into binary ratings of training set
training_set[training_set == 0] = -1 #all zeros to -1  
training_set[training_set == 1] = 0  #all 1 to 0
training_set[training_set == 2] = 0  #all 2 to 0  
training_set[training_set >= 3] = 1 #num>2 = 1     

#Converting ratings into binary ratings of test set
test_set[test_set == 0] = -1 #all zeros to -1  
test_set[test_set == 1] = 0  #all 1 to 0
test_set[test_set == 2] = 0  #all 2 to 0  
test_set[test_set >= 3] = 1 #num>2 = 1     

We have to create a class with three functions for the Restricted Boltzmann
Machine which it will obey.
        

1. Initialise tensors of all weights and biases of the  visible nodes and
   hidden nodes. Add weight parameter of the probabilities of the visible 
   nodes according to the hidden nodes.
2. Sample hidden nodes
   For every each hidden node activate them for a given probablity given v.
   In which the activation is a linear function of the neurons where the 
   coefficients are the functions. So, the activation is probability that the
   hidden node will be activated according to the value of the visible node. 
   The activation is returned as a sigmoid function. But we're making a 
   Bernoulli RBM. p[h|v] is vector of nh elements, each element corresponds to 
   each hidden node. We use this probabilities to sample activation of each 
   hidden node, depending on p[h|v] for v. If randn < 0.7 = activate neuron, 
   and if randn > 0.7 = not activate neuron. Obtain vector with a binary outcome 
   to list which hidden nodes activated or not activated.
3. Sample visible nodes.
   If randn < 0.25 = activate neuron, 
   and if randn > 0.25 = not activate neuron. Obtain vector with a binary outcome 
   to list which hidden nodes activated or not activated.

In [13]:
#Creating the architecture of the Neural Network
class RBM():
    def __init__(self, nv, nh):   
        self.W = torch.randn(nh,nv) ##add weight parameter of the probabilities of the visible/hidden nodes
        self.a = torch.randn(1, nh) ##bias for hidden nodes, additional element corresponding to current batch 
        self.b = torch.randn(1, nv) ##bias for visible nodes, additional element corresponding to current batch 
        
    def sample_h(self, x): #x is visible neuron's v in the probabilites p[h|v] = sigmoid[wx*a] 
        wx = torch.mm(x, self.W.t()) #product of tensor (nv,nh) * p[h|v]
        activation = wx + self.a.expand_as(wx) # wx++bias  
        p_h_given_v = torch.sigmoid(activation) #sigmoid of the activation = #i'th vector gives probability of hidden node activation for i'th vector
        return p_h_given_v, torch.bernoulli(p_h_given_v)  #Bernoulli samples
    
    def sample_v(self, y): #probabilities visible node = 1 given probablilites hidden nodes p[v|h] 
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy) # +bias of visible nodes  
        p_v_given_h = torch.sigmoid(activation)  
        return p_v_given_h, torch.bernoulli(p_v_given_h)  #Bernoulli sampling
    
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t() #update weights
        self.b += torch.sum((v0-vk), 0) #update bias for visible nodes
        self.a += torch.sum((ph0-phk), 0) #update bias for hidden nodes


KL-divergence to approximate loglikelikhood gradient. RBM-model is an energy
function that depends on the weights of the tensors we try to minimize. This 
is a probabliblistic graphical model to maximise the loglikelihood and minimise
the energy of the energy state. Therefore we need to compute the gradient, which is computing 
demanding. So we approximate the gradient with Gibbs sampling as following :
    1. Input vector V[0]
    2. Based on probabilities p[h|0] we sample hidden nodes = h1
    3. We sample visible nodes with activation p[v|h1] = v1
    4. Sample hidden nodes with activation p[h1|v1] = h2
    5. k-times.... 

Contrasted convergence algorithm - Each round is 1 Gibbs sample (Gibbs Sampling)

In [14]:
nv = len(training_set[0]) #first line of training set with x features = # of number of visible nodes
nh = 100 #Number of preferred features out of all 'movies'
batch_size = 100 #update weights after nth round
rbm = RBM(nv,nh) #Create model based on architecture


Model now created.


In [15]:
### Part - 3 Train the Restricted Boltzman Machine
epochs = 10 #Binary outcome and 934 observations
for epoch in range(1, epochs +1): #+1 because upper bound not included
    train_loss = 0
    s = 0. #Normalise the train loss by dividing loss w/ counter s 
    for id_user in range(0, num_users - batch_size, batch_size): #Users per batch 
        vk = training_set[id_user:id_user+batch_size] #Input that gets updated, vector output of k steps
        v0 = training_set[id_user:id_user+batch_size] #Target variable to calculate loss
        #p(movie = rating 1| rating of consumer)
        ph0,_ = rbm.sample_h(v0) #p(hidden node=1|target rating)
        
        #Loop model over k iterations for convergence
        for k in range(10):
            _,hk = rbm.sample_h(vk) #p(hidden node|visible node)
            _,vk = rbm.sample_v(hk) #p(visible node|hidden node) k = 10
            
            #Approximate the gradient to update weights and biases with vk
            vk[v0<0] = v0[v0<0] 
        #Compute phk --> Class;train(parameters)
        phk,_ = rbm.sample_h(vk) #apply to last step of visible node sampling
        #Train the Restricted Boltzmann Machine
        rbm.train(v0, vk, ph0, phk)
        #Update the train loss
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0])) #Include only existing ratings in training
        #Add +1 for normaliser
        s += 1.
    print('epoch: '+str(epoch) +'loss: '+str(train_loss/s))
        

epoch: 1loss: tensor(0.3548)
epoch: 2loss: tensor(0.2550)
epoch: 3loss: tensor(0.2468)
epoch: 4loss: tensor(0.2515)
epoch: 5loss: tensor(0.2484)
epoch: 6loss: tensor(0.2494)
epoch: 7loss: tensor(0.2495)
epoch: 8loss: tensor(0.2461)
epoch: 9loss: tensor(0.2479)
epoch: 10loss: tensor(0.2462)


Restricted Boltzmann Machine now trained. 3 out of 4 times we make a good prediction for the next movie rating for a user in the training set.

In [16]:
### Part - 4 Test the Restricted Boltzman Machine
test_loss = 0
s = 0. #Normalise the train loss by dividing loss w/ counter s 
for id_user in range(num_users): #Users per batch 
    v = training_set[id_user:id_user+1] #Vector input of output RBM on test_set 
    vt = test_set[id_user:id_user+1] #Target variable to calculate loss
             
    #Predict next energy state 
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v) #p(hidden node|visible node) - Sample hidden node
        _,v = rbm.sample_v(h) #p(visible node|hidden node) - Sample visible node
        #Update the train loss
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0])) #Avg.Distance
        #Add +1 for normaliser
        s += 1.
print('test loss: '+str(test_loss/s))


test loss: tensor(0.2692)


In [17]:
train_loss

tensor(2.2158)

In [18]:
test_loss

tensor(123.5720)

In [19]:
import numpy as np
u = np.random.choice([0,1], 100000)
v = np.random.choice([0,1], 100000)
u[:50000] = v[:50000]
sum(u==v)/float(len(u)) # -> you get 0.75
np.mean(np.abs(u-v)) # -> you get 0.25

0.25118